<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice Using APIs

_Authors: Dave Yerrington (SF), Sam Stack(DC)_

---

## Exercise: IMDB TV Shows

---

In this lab we will practice using some popular APIs to retrieve and store data.

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.

Here we will use a combination of scraping and API calls to find the ratings and networks of famous television shows.

In [1]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt

%matplotlib inline

### 3.A Get the top TV Shows

The Internet Movie Database contains data about movies and TV shows. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2 contains the list of the top 250 tv shows of all time. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

> **Hint:** movie_ids look like this: `tt2582802`
> _Everything after "/title/" and before "/?"_

In [2]:
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2')
    html = response.text
    # use greedy version to find everything after title to the next backslash in the a href element
    entries = re.findall("<a href.*?/title/(.*?)/", html) 
    # create a list of the top 250 results
    return list(set(entries))

In [3]:
entries = get_top_250()

In [4]:
len(entries)

251

In [5]:
entries[0]

'tt0112130'

### 3.B Get data on the top movies

Although the Internet Movie Database does not have a public API, an open API exists at http://www.tvmaze.com/api.

Use this API to retrieve information about each of the 250 TV shows you have extracted in the previous step.
1. Check the documentation of tvmaze's api to learn how to request show data by id.
- Define a function that returns a python object with select information for a given id.
    - Show name
    - Rating (avg)
    - Genre(s)
    - Network name
    - Premiere date
    - Status
> Tip: the json object can easily be converted into a python dictionary.

- Store the gathered information in a Pandas Dataframe.


As the target information is in json format you will need `json.loads(res.text)` in order to gather it.

Heres and example of the information and how we can interact with it.

In [6]:
# example url
res = requests.get('http://api.tvmaze.com/lookup/shows?imdb=tt0944947')

# status code
print(res.status_code)

# just the contents of the name element
print(json.loads(res.text).get('name'))

# entire contents
print(json.loads(res.text))

200
Game of Thrones
{'id': 82, 'url': 'http://www.tvmaze.com/shows/82/game-of-thrones', 'name': 'Game of Thrones', 'type': 'Scripted', 'language': 'English', 'genres': ['Drama', 'Adventure', 'Fantasy'], 'status': 'Running', 'runtime': 60, 'premiered': '2011-04-17', 'officialSite': 'http://www.hbo.com/game-of-thrones', 'schedule': {'time': '21:00', 'days': ['Sunday']}, 'rating': {'average': 9.4}, 'weight': 99, 'network': {'id': 8, 'name': 'HBO', 'country': {'name': 'United States', 'code': 'US', 'timezone': 'America/New_York'}}, 'webChannel': {'id': 22, 'name': 'HBO Go', 'country': {'name': 'United States', 'code': 'US', 'timezone': 'America/New_York'}}, 'externals': {'tvrage': 24493, 'thetvdb': 121361, 'imdb': 'tt0944947'}, 'image': {'medium': 'http://static.tvmaze.com/uploads/images/medium_portrait/143/359013.jpg', 'original': 'http://static.tvmaze.com/uploads/images/original_untouched/143/359013.jpg'}, 'summary': '<p>Based on the bestselling book series <i>A Song of Ice and Fire</i> 

In [7]:
#  function to pull information from API using Json interaction
def get_entry(entry):
    res=requests.get('http://api.tvmaze.com/lookup/shows?imdb='+entry)
    if res.status_code == 200:
        try:
            status = json.loads(res.text).get('status')
        except AttributeError:
            status = 'NA'
        try: 
            rating = json.loads(res.text).get('rating').get('average')
        except AttributeError:
            rating = 'NA'
            
        try:
            network = json.loads(res.text).get('network').get('name')
        except AttributeError:
            network = 'NA'
            
        try:
            title = json.loads(res.text).get('name')
        except AttributeError:
            title = 'NA'
            
        try:
            premier = json.loads(res.text).get('premiered')
        except AttributeError:
            premier = 'NA'
            
        try:
            genres = json.loads(res.text).get('genres')
        except AttributeError:
            genres = 'NA'

        # takes local variables as a 
        shows_df.loc[len(shows_df)] = [title, rating, genres, network, premier, status]

In [8]:
#  function to pull information from API converting Json into a python dictionary element
def get_entry(entry):
    res=requests.get('http://api.tvmaze.com/lookup/shows?imdb='+entry)
    if res.status_code == 200:
        results = json.loads(res.text)
        
        try:    
            status = results['status']
        except TypeError:
            status = 'NA'   
        try:
            rating = results['rating']['average']
        except TypeError:
            rating = 'NA'
        try:
            network = results['network']['name']
        except TypeError:
            network = 'NA'
        try:   
            title = results['name']
        except TypeError:
            title = 'NA'
        try:   
            genres = results['genres']
        except TypeError:
            genres = 'NA'
        try:   
            premier = results['premiered']
        except TypeError:
            premier = 'NA'
        
        shows_df.loc[len(shows_df)] = [title, rating, genres, network, premier, status]

In [9]:
# in both functions we are looking for specific elements.  If an element is missing an error will return thus the need
# for try and except statements.

In [ ]:
shows_df = pd.DataFrame(columns = ['show_name', 'rating_avg', 'genres', 'network', 'premiere_date', 'status'])

for entry in entries:
    get_entry(entry)

In [ ]:
shows_df.head()

In [ ]:
shows_df.info()